<a href="https://colab.research.google.com/github/zhackerx/Nlp/blob/master/nlpda2ques1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -U spacy 
!pip install sklearn_crfsuite

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)
     |████████████████████████████████| 747kB 4.0MB/s 


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [0]:
df = pd.read_csv('/content/ner_dataset.csv', encoding = "ISO-8859-1")

In [9]:
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [10]:
df.describe()

,Sentence #,Word,POS,Tag
count,47959,1048575,1048575,1048575
unique,47959,35178,42,17
top,Sentence: 221,the,NN,O
freq,1,52573,145807,887908


In [0]:
df['Tag'].unique()
df.isnull().sum()
df = df.fillna(method = 'ffill')

In [0]:
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [41]:
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]
sent[0][1]

'NNS'

In [14]:
sent = getter.get_text()
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [0]:
def word2features(sent, i):
    print(sent[0][1])
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [66]:
for s in sentences:
  res = len(s.split())
  print(res)

Streaming output truncated to the last 5000 lines.
17
20
15
13
13
27
20
27
24
27
25
27
28
23
15
7
15
21
14
29
27
26
24
17
20
16
14
19
24
20
7
23
21
20
20
22
19
11
30
40
31
7
19
20
26
18
21
24
16
14
34
47
29
23
14
31
21
27
24
24
19
23
32
28
15
18
25
17
16
14
17
12
27
26
27
21
13
14
11
7
19
6
31
21
34
23
15
13
18
27
9
16
25
12
16
33
7
15
22
9
19
21
6
31
29
30
28
20
32
17
19
20
24
24
13
8
27
12
18
19
15
15
24
43
30
21
14
22
20
32
12
13
26
33
24
38
27
50
35
24
22
22
11
24
29
25
11
19
14
13
13
40
12
9
26
11
20
25
19
28
21
37
25
13
11
10
23
31
18
9
19
29
40
31
21
25
39
24
31
20
12
13
12
23
17
23
23
24
26
21
19
27
16
31
19
16
19
19
25
17
42
16
17
17
23
27
30
25
9
17
7
25
29
30
25
13
10
22
29
26
11
15
24
27
28
30
39
12
26
28
18
11
26
21
36
30
11
15
18
16
14
32
25
36
20
13
12
27
27
19
19
28
24
35
10
28
31
27
27
25
32
37
38
25
16
28
26
27
29
14
34
15
38
13
17
28
25
6
35
21
20
17
16
26
15
11
9
22
25
19
11
30
22
17
20
14
28
18
15
13
31
16
21
18
26
12
21
18
24
18
20
14
20
25
24
11
11
31
14
18
15
21

In [0]:
sentences = getter.sentences

In [0]:
batch_size = 64 
# Passes through entire dataset
epochs = 8
# Maximum length of review
max_len = 75 
# Dimension of embedding vector
embedding = 40

In [0]:
words = list(df['Word'].unique())
tags = list(df['Tag'].unique())
# Dictionary word:index pair
# word is key and its value is corresponding index
word_to_index = {w : i + 2 for i, w in enumerate(words)}
word_to_index["UNK"] = 1
word_to_index["PAD"] = 0

# Dictionary lable:index pair
# label is key and value is index.
tag_to_index = {t : i + 1 for i, t in enumerate(tags)}
tag_to_index["PAD"] = 0

idx2word = {i: w for w, i in word_to_index.items()}
idx2tag = {i: w for w, i in tag_to_index.items()}

In [72]:
print("The word India is identified by the index: {}".format(word_to_index["India"]))
print("The label B-org for the organization is identified by the index: {}".format(tag_to_index["B-org"]))

The word India is identified by the index: 2570
The label B-org for the organization is identified by the index: 6


In [75]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-jk3n6fzp
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-jk3n6fzp
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=f0334dc3a044d3b29cc24abe589a35bdabd8b0c6307dcb45bdca2387b81b1385
  Stored in directory: /tmp/pip-ephem-wheel-cache-bsd7qs36/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [79]:
!pip install simpletransformers

     |████████████████████████████████| 194kB 2.9MB/s 
     |████████████████████████████████| 204kB 9.0MB/s 
     |████████████████████████████████| 665kB 7.9MB/s 
     |████████████████████████████████| 3.8MB 18.0MB/s 
     |████████████████████████████████| 890kB 43.3MB/s 
     |████████████████████████████████| 1.1MB 44.8MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=7f67756fe55ec9653087719044383fb197b6fc0103779110e1f12b81150c60c3
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=416eeee90dfb3674dd4397255af11e4f5145f169d865f4f2769ec23c9d323e5e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses


In [0]:
import pandas as pd
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
from keras.models import Model, Input
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import f1_score
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from keras.preprocessing.text import text_to_word_sequence
import pickle

In [0]:
# Converting each sentence into list of index from list of tokens
X = [[word_to_index[w[0]] for w in s] for s in sentences]

# Padding each sequence to have same length  of each word
X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = word_to_index["PAD"])

In [0]:
y = [[tag_to_index[w[2]] for w in s] for s in sentences]

# padding
y = pad_sequences(maxlen = max_len, sequences = y, padding = "post", value = tag_to_index["PAD"])

In [0]:
num_tag = df['Tag'].nunique()
# One hot encoded labels
y = [to_categorical(i, num_classes = num_tag + 1) for i in y]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15)

In [86]:
print("Size of training input data : ", X_train.shape)
print("Size of training output data : ", np.array(y_train).shape)
print("Size of testing input data : ", X_test.shape)
print("Size of testing output data : ", np.array(y_test).shape)

Size of training input data :  (40765, 75)
Size of training output data :  (40765, 75, 18)
Size of testing input data :  (7194, 75)
Size of testing output data :  (7194, 75, 18)


In [88]:
num_tags = df['Tag'].nunique()
# Model architecture
input = Input(shape = (max_len,))
model = Embedding(input_dim = len(words) + 2, output_dim = embedding, input_length = max_len)(input)
model = Bidirectional(LSTM(units = 50, return_sequences=True, recurrent_dropout=0.1))(model)
model = TimeDistributed(Dense(50, activation="relu"))(model)
crf = CRF(num_tags+1)  # CRF layer
out = crf(model)  # output

model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 75)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 75, 40)            1407200   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 75, 100)           36400     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 75, 50)            5050      
_________________________________________________________________
crf_2 (CRF)                  (None, 75, 18)            1278      
Total params: 1,449,928
Trainable params: 1,449,928
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpointer = ModelCheckpoint(filepath = 'model.h5',
                       verbose = 0,
                       mode = 'auto',
                       save_best_only = True,
                       monitor='val_loss')

In [90]:
history = model.fit(X_train, np.array(y_train), batch_size=batch_size, epochs=epochs,
                    validation_split=0.1, callbacks=[checkpointer])

Train on 36688 samples, validate on 4077 samples
Epoch 1/8
36688/36688 [==============================] - 105s 3ms/step - loss: 0.1574 - crf_viterbi_accuracy: 0.9588 - val_loss: 0.0447 - val_crf_viterbi_accuracy: 0.9845
Epoch 2/8
36688/36688 [==============================] - 102s 3ms/step - loss: 0.0243 - crf_viterbi_accuracy: 0.9883 - val_loss: 0.0155 - val_crf_viterbi_accuracy: 0.9892
Epoch 3/8
36688/36688 [==============================] - 109s 3ms/step - loss: 0.0060 - crf_viterbi_accuracy: 0.9909 - val_loss: 0.0036 - val_crf_viterbi_accuracy: 0.9901
Epoch 4/8
36688/36688 [==============================] - 103s 3ms/step - loss: -0.0059 - crf_viterbi_accuracy: 0.9919 - val_loss: -0.0065 - val_crf_viterbi_accuracy: 0.9903
Epoch 5/8
36688/36688 [==============================] - 102s 3ms/step - loss: -0.0160 - crf_viterbi_accuracy: 0.9925 - val_loss: -0.0145 - val_crf_viterbi_accuracy: 0.9901
Epoch 6/8
36688/36688 [==============================] - 102s 3ms/step - loss: -0.0253 - crf

In [0]:
# Evaluation
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=-1)
y_test_true = np.argmax(y_test, -1)

In [0]:
y_pred = [[idx2tag[i] for i in row] for row in y_pred]
y_test_true = [[idx2tag[i] for i in row] for row in y_test_true]

In [93]:
print("F1-score is : {:.1%}".format(f1_score(y_test_true, y_pred)))

F1-score is : 88.5%


In [94]:
report = flat_classification_report(y_pred=y_pred, y_true=y_test_true)
print(report)

              precision    recall  f1-score   support

       B-art       0.25      0.02      0.04        48
       B-eve       0.45      0.20      0.28        50
       B-geo       0.86      0.89      0.87      5674
       B-gpe       0.95      0.94      0.95      2380
       B-nat       0.54      0.24      0.33        29
       B-org       0.80      0.70      0.75      3051
       B-per       0.82      0.84      0.83      2530
       B-tim       0.92      0.89      0.90      3102
       I-art       0.25      0.03      0.05        35
       I-eve       0.08      0.02      0.03        51
       I-geo       0.79      0.79      0.79      1088
       I-gpe       0.74      0.50      0.60        28
       I-nat       0.00      0.00      0.00        10
       I-org       0.78      0.77      0.78      2469
       I-per       0.87      0.85      0.86      2566
       I-tim       0.85      0.74      0.79      1011
           O       0.99      0.99      0.99    133505
         PAD       1.00    

In [95]:
i = np.random.randint(0,X_test.shape[0]) # choose a random number between 0 and len(X_te)b
p = model.predict(np.array([X_test[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_test[i], -1)

print("Sample number {} of {} (Test Set)".format(i, X_test.shape[0]))
# Visualization
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_test[i], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-2], idx2tag[t], idx2tag[pred]))

Sample number 4256 of 7194 (Test Set)
Word           ||True ||Pred
The            : O     O
attack         : O     O
follows        : O     O
the            : O     O
assassination  : O     O
of             : O     O
four           : O     O
French         : B-gpe B-gpe
tourists       : O     O
on             : O     O
Monday         : B-tim B-tim
in             : O     O
eastern        : B-geo B-geo
Mauritania     : I-geo I-geo
.              : O     O
